In [ ]:
%load_ext autoreload
%autoreload 2

def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))

In [ ]:
from simsiam_train import dataio_prep
import torch
import speechbrain as sb

In [ ]:
hparams = {
    'sample_rate': 16000,
}
label_encoder_path = './dataset/label_encoder.txt'
label_encoder = sb.dataio.encoder.CategoricalEncoder()
label_encoder.load_or_create(label_encoder_path)

csv_path = '/mnt/data/zhepei/outputs/cssl_sound/results/2022-02-05+06-01-38_seed_1234+offline/save/train_task0_replay.csv'
ds = dataio_prep(
    hparams,
    csv_path,
    label_encoder
)

In [ ]:
from speechbrain.lobes import augment
time_domain_aug = augment.TimeDomainSpecAugment(
    sample_rate=hparams['sample_rate'],
    speeds=[95, 100, 105],
    drop_freq_count_high=4,
    drop_chunk_count_high=3,
    drop_chunk_length_low=1000,
    drop_chunk_length_high=8000,
    drop_chunk_noise_factor=0.,
#     drop_chunk_count_high=2,
#     drop_chunk_noise_factor=0.05,
)

In [ ]:
def compute_aug(wavs):
    lens = torch.ones(wavs.shape[0]).to(wavs.device)
    wavs_aug = time_domain_aug(wavs, lens)
    if wavs_aug.shape[1] > wavs.shape[1]:
        wavs_aug = wavs_aug[:, 0 : wavs.shape[1]]
    else:
        zero_sig = torch.zeros_like(wavs)
        zero_sig[:, 0 : wavs_aug.shape[1]] = wavs_aug
        wavs_aug = zero_sig
    return wavs_aug




In [ ]:
x = ds[1211]['sig'].unsqueeze(0)
x1 = compute_aug(x)
x2 = compute_aug(x)

In [ ]:
sound(x[0].numpy(), 16000, label='orig')
sound(x1[0].numpy(), 16000, label='x1')
sound(x2[0].numpy(), 16000, label='x2')